In [5]:
# pip install sentence_transformers
# pip install -U scikit-learn
# pip install rank_bm25

In [1]:
import torch
from transformers import (
    AdamW,
    AutoModel,
    get_linear_schedule_with_warmup,
    AutoTokenizer,
    AutoConfig
)

In [2]:
from skt.vault_utils import get_secrets
proxies = get_secrets('proxies')

In [3]:
import os 
import numpy as np
import torch

In [4]:
os.environ['http_proxy'] = proxies['http']
os.environ['https_proxy'] = proxies['https']

In [5]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [6]:
from src.utils import set_seed
from src.trainer import SimcseTrainer
from src.dataset import DATASET_MAPPING_DICT
from src.utils import PreprocessorFactory 
from src.utils import get_model_argparse
from src.model import MODEL_MAPPING_DICT
from src.model import CONFIG_MAPPING_DICT
from src.logger import Experi_Logger
from config.nli_config import nli_parser_model_args

In [7]:
#base_model_url = "jhlee3421/faq-semantic-klue-roberta-large"
#base_model_url= 'klue/roberta-large'

In [8]:
args = nli_parser_model_args()

In [9]:
args.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
args.n_gpu = torch.cuda.device_count()
args.output_dir = f'/home/x1112436/result/faq/modelfile/{args.pretrained_model}'
args.log_dir = f'/home/x1112436/result/faq/log/{args.pretrained_model}'
args.experiments_path = f'/home/x1112436/result/faq/experiment/{args.pretrained_model}/experiment.csv'
args.is_preprocessed = True
args.valid_first = False
args.data_type='triple'
args.loss= 'TripletLoss'
args.margin = 0.7
args.num_train_epochs = 20
args.model_max_len = 50
args.valid_first = True

# Load model

In [10]:
args.pretrained_model = '/home/x1112436/model_file/faq_sent_roberta/sent_roberta'

In [11]:
model = MODEL_MAPPING_DICT['sent_roberta'].from_pretrained(
    args.pretrained_model, **vars(args), 
)
tokenizer = AutoTokenizer.from_pretrained(args.pretrained_model)

# define function

In [12]:
# chunking inetent
def chunking(input_list, chunk_size=200):
    if len(input_list) < 200:
        return [input_list]
    
    result = []
    chunk_length = int(len(input_list) / chunk_size)
    chunk_rest = len(input_list) % chunk_size

    start = 0
    for i in range(0, chunk_length):
        result.append(input_list[start: start + chunk_size])
        start = start + chunk_size
                      
    if chunk_rest > 0 :
        result.append(input_list[start:])
    return result

In [13]:
print(device)

cuda


# Load DATA

In [1]:
from skt.gcp import load_bigquery_ipython_magic, \
                    bq_to_pandas, \
                    get_bigquery_client

In [8]:
dataset = 'x1112436'
log_table = 'faq_table'
query = f"""

SELECT  query,
        answer,
        intent_nm,
        answer,
        domain,
        status
FROM `skt-datahub.{dataset}.{log_table}`
WHERE intent_nm !='' and intent_nm is not null
"""

In [9]:
faq_table = bq_to_pandas(query)

query: 

SELECT  query,
        answer,
        intent_nm,
        answer,
        domain,
        status
FROM `skt-datahub.x1112436.faq_table`
WHERE intent_nm !='' and intent_nm is not null

destination: skt-datahub._775c5ccab1096b3cccd7ac34a5db11c0a354fb07.anon7a2dc238204be3628489f97c3c6db67a688b5c35ec64f5d6524dbb912d4fe000
total_rows: 246427
slot_secs: 1.758

Downloading: 100%|██████████|


In [15]:
faq_table.loc[faq_table.answer==''].head(2)             

,query,answer,intent_nm,answer_1,domain,status
3,baro 3GB 문의,,baro 3GB,,sms_customer_center,INSERT
4,baro 3GB,,baro 3GB,,sms_customer_center,INSERT


# blacklist 처리

In [262]:
idx2query = list(faq_table.qry_txt_cont.unique())
idx2intent_nm = list(faq_table.intent_nm.unique())
idx2intent_nm_origin = list(faq_table.intent_nm_orgin.unique())

In [263]:
with open('intent_nm.csv', 'a') as f:
    for intent, origin in zip(idx2intent_nm, idx2intent_nm_origin):
        f.write(intent + '\t' + origin + '\n')

In [192]:
faq_table_q_a = faq_table[['qry_txt_cont', 'intent_nm']]
query_to_answer = faq_table_q_a.set_index('qry_txt_cont').to_dict()['intent_nm']

# Token Count

In [193]:
from collections import Counter, OrderedDict

In [194]:
intent_list = list(faq_table.intent_nm.unique())
query_list = list(faq_table.qry_txt_cont.unique())

In [195]:
query_token_list = []
for query in query_list:
    query_token_list.extend(query.split())

In [196]:
intent_token_list = []
for intent in intent_list:
    intent_token_list.extend(intent.split())

In [197]:
intent_token_list = OrderedDict(Counter(intent_token_list).most_common())

In [198]:
query_token_list = OrderedDict(Counter(query_token_list).most_common())

# SAVE DATA FOR EACH LABEL (INTENT_NM)

In [100]:
#faq_table.loc[faq_table.intent_nm == intent_list[209]].iloc[0]          

In [99]:
# for i, intent in enumerate(intent_list):
#     if intent.startswith('T전화 프로필을 설정하면'):
#         print(i)
#         break

In [126]:
for i, intent in enumerate(intent_list):
    file_name = f'./data/label_data/{i}.csv'
    queries = list(faq_table.loc[faq_table.intent_nm == intent]['qry_txt_cont'].unique())
    intent = intent.replace("'", "")
    with open(file_name, 'a+') as f:
        f.write(intent + '\n')
        for query in queries:
            f.write(query + '\n')

# one sentence embedding setting

In [83]:
from torch.utils.data import (
    DataLoader, Dataset
)

In [84]:
from dataclasses import dataclass
from typing import List, Any, Union, Dict

In [85]:
@dataclass
class SingleSentenceInput:
    sentence_a: str = None
    a_input_ids: List[int] = None
    a_attention_mask: List[int] = None

In [86]:
class EmbeddingDataset(Dataset):
    def __init__(
            self,
            args,
            features:List[SingleSentenceInput],
            max_length,
            tokenizer,
            **kwargs
    ):
        super(EmbeddingDataset, self).__init__()
        self.args = args
        self.features = features
        self.max_length = max_length
        self.pad_token_id = tokenizer.pad_token_id
        self.sep_token_id = tokenizer.sep_token_id if tokenizer.sep_token_id else tokenizer.eos_token_id

    def __getitem__(self, index) -> Dict[str, Any]:
        feature = self.features[index]
        return {
            'a_sentence': feature.sentence_a,
            'a_input_ids': torch.tensor(feature.a_input_ids, dtype=torch.long),
            'a_attention_mask': torch.tensor(feature.a_attention_mask, dtype=torch.long)
        }
    def __len__(self):
        return len(self.features)
    
    def loader(self, shuffle:bool=True, batch_size:int=64):
        return DataLoader(self, shuffle=shuffle, batch_size=batch_size, collate_fn=self.collater)

    def collater(self, batch: List[Dict[str, Any]]) -> Dict[str, Any]:

        a_sentence = [data['a_sentence'] for data in batch]
        a_input_ids = [data['a_input_ids'] for data in batch]
        a_attention_mask = [data['a_attention_mask'] for data in batch]
        ##  token level encoding
        batch_size = len(batch)
        sizes = [len(s) for s in a_input_ids]
        target_size = min(max(sizes), self.max_length)
        """ torch.full -> creates a tensor of a given shape and fills it with a scalar value self.pad_token_id here"""
        a_collated_ids = torch.full((batch_size, target_size), self.pad_token_id, dtype=torch.long)
        a_collated_attention_masks = torch.zeros((batch_size, target_size), dtype=torch.long)

        """ cut data if size > target_size else: fill by self.pad_token_id """
        for i, (input_id, attention_m, size) in enumerate(
                zip(a_input_ids, a_attention_mask, sizes)):
            diff = target_size - size
            if diff < 0:
                a_collated_ids[i, :target_size] = input_id[:target_size]
                a_collated_ids[i, -1] = self.sep_token_id
                a_collated_attention_masks[i, :target_size] = attention_m[:target_size]

            else:
                a_collated_ids[i, :size] = input_id
                a_collated_attention_masks[i, :size] = attention_m

        return {
            'a_sentence': a_sentence,
            'a_input_ids': a_collated_ids,
            'a_attention_mask': a_collated_attention_masks
        }

In [88]:
from src.utils.abs_preprocess import AbsPreprocessor

class Testprocessor(AbsPreprocessor):

    @classmethod
    def preprocess(cls, tokenizer,  input_list:List) -> None:
        """ try read tsv file using pandas first if [memory or parse] error catched use other reading method  """
    
        feature_list = list()
        skipped_line = 0

        for i, line in enumerate(input_list):
            try:
                a_encoded_sentence = cls.tokenizing(input=line, tokenizer=tokenizer, tokenizer_input=None)
                feature_list.append(
                    SingleSentenceInput(
                        sentence_a = line,
                        a_input_ids = a_encoded_sentence.input_ids,
                        a_attention_mask=a_encoded_sentence.attention_mask,
                    )
                )
            except Exception as e:
                print(f'Error occurs in {i} lines in preprocessing')
                print(line)
                print(e)
                break

        return feature_list


# Kmeans sampling for Representative query for each intent_nm

In [89]:
model = model.to(device)

NameError: name 'model' is not defined

In [ ]:
def encode(chunk_list):
    emedding_dict = dict()
    embedding_list = []
    query_list = []
    for chunk in chunk_list:
        chunk_process = Testprocessor.preprocess(tokenizer = tokenizer, input_list = chunk)
        chunk_dataset = EmbeddingDataset(args=args, features=chunk_process, max_length=args.model_max_len, tokenizer=tokenizer)
        chunk_dataloader = chunk_dataset.loader(shuffle=False, batch_size=400)
        model.eval()
        with torch.no_grad():   
            for batch_idx, batch in enumerate(chunk_dataloader): 
                batch = {key: (item.to(args.device) if type(item) == torch.Tensor else item) for key, item in batch.items()}
                a_embedding = model(batch['a_input_ids'], batch['a_attention_mask'])
                a_sentence = batch['a_sentence']
                query_list.extend(a_sentence)
                embedding_list.append(a_embedding)
            
    embeddings = torch.cat(embedding_list, 0) 
    return embeddings, query_list


In [90]:
embedding_dict = {}
query_dict = {}
for i, intent in enumerate(idx2intent_nm):
    if i % 400 ==0:
        print(i)
        
    unique_query = faq_table_q_a[faq_table_q_a.intent_nm == intent]
    candidate_query = list(unique_query.qry_txt_cont.unique())
    model_input_list = chunking(candidate_query, chunk_size=300)
    embeddings, queries = encode(model_input_list)    
    embedding_dict[intent] = embeddings.cpu().numpy()
    query_dict[intent] = queries

0


NameError: name 'chunking' is not defined

In [83]:
import pickle
with open('./embedding_result/intent_query_embedding_klue_roberta_large.pkl', 'wb') as f:
    pickle.dump({
        'query_dict': query_dict,
        'embedding_dict': embedding_dict
    }, f)

# after embedding

In [1]:
import pickle
with open('./embedding_result/intent_query_embedding_klue_roberta_large.pkl', 'rb') as f:
    test = pickle.load(f)
 

In [2]:
query_dict = test['query_dict']
embedding_dict = test['embedding_dict']

In [3]:
import sklearn
from sklearn.cluster import KMeans
import numpy as np

In [4]:
def get_cluster_size(data_size):
    if data_size >= 100:
        cluster_size = 10
    elif (data_size < 100) and (data_size > 30) :
        cluster_size = 5
    else:
        cluster_size = 1
    return cluster_size  

In [5]:
# pip install -U scikit-learn

In [6]:
from sklearn.cluster import KMeans

In [7]:
from tqdm.notebook import tqdm

In [8]:
# len(query_dict[test[0][0]])

In [9]:
from collections import defaultdict
sample_prop = 0.4
rep_queries = defaultdict(list)
neg_rep_queries = defaultdict(list)

for key, value in tqdm(list(embedding_dict.items())):
    intent_nm = key
    query = query_dict[intent_nm]
    query = [j for sub in query for j in sub]
    data_size = value.shape[0]
    cluster_size = get_cluster_size(data_size)
    sampled_q = []
    neg_sampled_q = []
    if cluster_size > 1:
        kmeans_label = KMeans(n_clusters=cluster_size, random_state=0, n_init="auto", max_iter=1000, init='k-means++').fit_predict(value)
        for i in range(cluster_size):
            c_i = np.where(kmeans_label == i)[0].tolist() 
            n_i = len(c_i)
            sample_i = np.random.choice(c_i, max(int(sample_prop * n_i), 1), replace=False)
            train_sample = sample_i[: int(0.8 * len(sample_i))]
            val_sample = sample_i[int(0.8 * len(sample_i)):]
            sampled_q.extend([query[ind] for ind in train_sample])
            neg_sampled_q.extend([query[ind] for ind in val_sample])
        rep_queries[intent_nm] = sampled_q
        neg_rep_queries[intent_nm] = neg_sampled_q
    else:
        if len(query) > 2:
            ind = int(np.ceil(1/2 * len(query)))
            rep_queries[intent_nm] = query[:ind]
            neg_rep_queries[intent_nm] = query[ind:]
        else:
            rep_queries[intent_nm] = query
            neg_rep_queries[intent_nm] = []

  0%|          | 0/1606 [00:00<?, ?it/s]

# SAVE KMEANS 

In [10]:
import pickle
with open('./embedding_result/kmeans_result.pkl', 'wb') as f:
    pickle.dump({
        'train': rep_queries,
        'test': neg_rep_queries
    }, f)

# Hard negative sampling based on BM25

In [11]:
train_query_list = []
for key, val in rep_queries.items():
    train_query_list.extend(val)

In [12]:
len(train_query_list)

81567

In [32]:
#!pip install rank_bm25

In [108]:
from rank_bm25 import BM25Okapi

In [15]:
intent_list = []
for key in query_dict.keys():
    intent_list.append(key)

In [199]:
def tokenizer_white(sent):
  sent = sent.replace("'", "")
  return sent.split(" ")

In [211]:
tokenized_corpus = [tokenizer_white(intent) for intent in intent_list]

In [212]:
bm25 = BM25Okapi(tokenized_corpus)

In [213]:
intent_list[:5]

['자동안심 T로밍 음성', '로밍 통화 요금', 'baro Box 임대료', 'T로밍청구내역서', '로밍중USIM변경']

In [229]:
negative_dict = dict()
negative_count = 5
for intent in intent_list:
    intent = intent.replace("'", "")
    neg_list = []
    try:
        negatives = bm25.get_top_n(tokenizer_white(intent), intent_list, n = negative_count + 2)[1:]
    except Exception as e:
        print(intent)
        print(e)
        break
        
    for neg in negatives:
        if (neg in intent) or (intent in neg):
            pass
        else:
            neg_list.append(neg)
    negative_dict[intent] = neg_list

In [230]:
negative_dict

{'자동안심 T로밍 음성': ['자동안심 T로밍 데이터',
  '자동안심 T로밍 데이터 해지',
  'T로밍 음성 사용량 안내 문자 차단',
  'T로밍 콜키퍼',
  'T로밍 도착알리미'],
 '로밍 통화 요금': ['데이터 로밍 요금',
  '로밍 수신 요금',
  '로밍 문자 요금',
  '미성년자 로밍',
  '기타 요금',
  '통신서비스 요금'],
 'baro Box 임대료': ['baro Box 잔여량 조회',
  'baro Box 임대 예약 방법',
  '로밍폰 임대료',
  'baro YT 요금제',
  'baro 요금제 잔여량 확인'],
 'T로밍청구내역서': ['0청년5960GB업',
  'ZEM',
  '골드번호',
  '골드번호프로모션',
  '부가서비스 가입내역 확인 방법',
  'T전화 그룹통화'],
 '로밍중USIM변경': ['0청년5960GB업',
  'ZEM',
  '골드번호',
  '골드번호프로모션',
  '부가서비스 가입내역 확인 방법',
  'T전화 그룹통화'],
 '로밍 중 전화 발신 방법': ['로밍 중 착신전환 사용',
  '로밍 중 국제전화카드 사용',
  '로밍 중 LTE 데이터 이용 방법',
  '로밍 중 인증 문자 수신',
  '국제전화 발신 금지',
  '미성년자 로밍'],
 'T로밍고객센터': ['0청년5960GB업',
  'ZEM',
  '골드번호',
  '골드번호프로모션',
  '부가서비스 가입내역 확인 방법',
  'T전화 그룹통화'],
 '데이터로밍차단시Wi-Fi이용': ['0청년5960GB업',
  'ZEM',
  '골드번호',
  '골드번호프로모션',
  '부가서비스 가입내역 확인 방법',
  'T전화 그룹통화'],
 '데이터로밍차단시통화문자이용': ['0청년5960GB업',
  'ZEM',
  '골드번호',
  '골드번호프로모션',
  '부가서비스 가입내역 확인 방법',
  'T전화 그룹통화'],
 'T끼리데이터선물하기받은데이터사용방법': ['0청년5960GB업',
  'ZEM',
  '골드번호

# SAVE negative dict and intent_list

In [130]:
import pickle
with open('./data/negative_dict.pkl', 'wb') as f:
    pickle.dump(negative_dict, f)

In [131]:
import pickle
with open('./data/intent_list.pkl', 'wb') as f:
    pickle.dump(intent_list, f)

# Add vocab based on frequency count

In [22]:
candidate_add_intent_token_list = []
candidate_add_query_token_list = []
add_token_count = 20

In [23]:
for key, val in intent_token_list.items():
    if key not in tokenizer.vocab and len(key) > 1:
        #candidate_add_intent_token_list.append("["+ key + "]")
        candidate_add_intent_token_list.append(key)
    if len(candidate_add_intent_token_list) == add_token_count:
        break
        

NameError: name 'intent_token_list' is not defined

In [50]:
save_path = './data/add_tokens.csv'
with open(save_path, 'a') as f:
    for data in candidate_add_intent_token_list:
        f.write(data + '\n')

In [47]:
print(candidate_add_intent_token_list)

['NUGU', '우주패스', 'T로밍', 'wavve', 'All케어플러스3', '안될', 'T전화', '컬러링', '휴대폰에서', 'T플랜', 'baro', '월정액', '고객센터', '비밀번호', 'T끼리', '5G', '카드/쿠폰', '선택약정', '할인제도', 'SMS']


In [50]:
word_embedding_model = models.Transformer(base_model_url)

In [51]:
word_embedding_model.tokenizer.add_tokens(candidate_add_intent_token_list, special_tokens=True)

20

In [56]:
word_embedding_model.tokenizer.model_max_length=128

In [57]:
word_embedding_model.auto_model.resize_token_embeddings(len(word_embedding_model.tokenizer))
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

In [58]:
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [59]:
model.max_seq_length = 128

# DATASET FOR TRAIN

In [60]:
from sentence_transformers import SentenceTransformer, SentencesDataset, losses
from sentence_transformers.readers import InputExample

In [61]:
print(negative_dict[intent_nm])
print(intent_nm)

['휴대폰 분실 정지 방법', '보이는 ARS 해제 방법', '골드번호', '골드번호프로모션', '부가서비스 가입내역 확인 방법', 'T전화 그룹통화', '분실 정지 해제 방법', '컬러링플러스2 신청 방법', '컬러링플러스2', '컬러링플러스2 설정 방법', '컬러링플러스2 해지 방법', 'ZEM(구. 쿠키즈)', 'T로밍 카드/쿠폰 고객센터', '5G 로밍 서비스']
0청년5960GB업


In [135]:
save_path = './data/train.csv'
train_dataset = []
for i, intent_nm in enumerate(intent_list):
    intent_nm = intent_nm.replace("'", "")
    queries = list(faq_table.loc[faq_table.intent_nm == intent_nm]['qry_txt_cont'].unique())
    for query in queries:
        for negative in negative_dict[intent_nm]:
            train_dataset.append([query, intent_nm, negative])

In [139]:
faq_table.loc[faq_table.intent_nm=='초시대 5G 휴대폰은 SK텔레콤과 함께!']

,qry_txt_cont,ans_cont,intent_nm_orgin,intent_nm,type
766,5G 기기변경 방법,초시대 5G 휴대폰은 SK텔레콤과 함께!\n국내 최고 속도의 5G 이용을 위해서는 ...,5G기기변경방법,초시대 5G 휴대폰은 SK텔레콤과 함께!,faq
23887,채티야 5G 휴대폰 기기변경 방법 알려줘,초시대 5G 휴대폰은 SK텔레콤과 함께!\n국내 최고 속도의 5G 이용을 위해서는 ...,5G기기변경방법,초시대 5G 휴대폰은 SK텔레콤과 함께!,faq
23888,채티야 여기서 5G 기변 문의 가능해?,초시대 5G 휴대폰은 SK텔레콤과 함께!\n국내 최고 속도의 5G 이용을 위해서는 ...,5G기기변경방법,초시대 5G 휴대폰은 SK텔레콤과 함께!,faq
23889,최신 5G 휴대폰 변경 원해요,초시대 5G 휴대폰은 SK텔레콤과 함께!\n국내 최고 속도의 5G 이용을 위해서는 ...,5G기기변경방법,초시대 5G 휴대폰은 SK텔레콤과 함께!,faq
23890,티월드 다이렉트 5G 핸드폰 구매 가능해요?,초시대 5G 휴대폰은 SK텔레콤과 함께!\n국내 최고 속도의 5G 이용을 위해서는 ...,5G기기변경방법,초시대 5G 휴대폰은 SK텔레콤과 함께!,faq
...,...,...,...,...,...
24052,5GX 전용 휴대폰 살래요,초시대 5G 휴대폰은 SK텔레콤과 함께!\n국내 최고 속도의 5G 이용을 위해서는 ...,5G기기변경방법,초시대 5G 휴대폰은 SK텔레콤과 함께!,faq
24053,스마트폰 5G로 변경할 수 있나요?,초시대 5G 휴대폰은 SK텔레콤과 함께!\n국내 최고 속도의 5G 이용을 위해서는 ...,5G기기변경방법,초시대 5G 휴대폰은 SK텔레콤과 함께!,faq
24054,지금 쓰는 요금제로 5G 단말기 사용 가능한가요?,초시대 5G 휴대폰은 SK텔레콤과 함께!\n국내 최고 속도의 5G 이용을 위해서는 ...,5G기기변경방법,초시대 5G 휴대폰은 SK텔레콤과 함께!,faq
24055,지금 요금제로 5G 스마트폰 사용 못 해?,초시대 5G 휴대폰은 SK텔레콤과 함께!\n국내 최고 속도의 5G 이용을 위해서는 ...,5G기기변경방법,초시대 5G 휴대폰은 SK텔레콤과 함께!,faq


In [138]:
train_dataset[:10]

[['5G 기기변경 방법',
  '초시대 5G 휴대폰은 SK텔레콤과 함께!',
  "'5G 휴대폰에서 5G 데이터 접속이 안될 때 조치 방법'에 대해 안내해드릴게요."],
 ['5G 기기변경 방법', '초시대 5G 휴대폰은 SK텔레콤과 함께!', "'5G T가족모아데이터'에 대해 안내해 드릴게요."],
 ['5G 기기변경 방법', '초시대 5G 휴대폰은 SK텔레콤과 함께!', "'5G 커버리지'에 대해 안내해 드릴게요."],
 ['5G 기기변경 방법', '초시대 5G 휴대폰은 SK텔레콤과 함께!', "'5G 클러스터'에 대해 안내해 드릴게요."],
 ['5G 기기변경 방법', '초시대 5G 휴대폰은 SK텔레콤과 함께!', "'5G ZEM플랜 퍼펙트'에 대해 안내해 드릴게요."],
 ['5G 기기변경 방법', '초시대 5G 휴대폰은 SK텔레콤과 함께!', "'5G ZEM플랜 베스트'에 대해 안내해 드릴게요."],
 ['채티야 5G 휴대폰 기기변경 방법 알려줘',
  '초시대 5G 휴대폰은 SK텔레콤과 함께!',
  "'5G 휴대폰에서 5G 데이터 접속이 안될 때 조치 방법'에 대해 안내해드릴게요."],
 ['채티야 5G 휴대폰 기기변경 방법 알려줘',
  '초시대 5G 휴대폰은 SK텔레콤과 함께!',
  "'5G T가족모아데이터'에 대해 안내해 드릴게요."],
 ['채티야 5G 휴대폰 기기변경 방법 알려줘',
  '초시대 5G 휴대폰은 SK텔레콤과 함께!',
  "'5G 커버리지'에 대해 안내해 드릴게요."],
 ['채티야 5G 휴대폰 기기변경 방법 알려줘',
  '초시대 5G 휴대폰은 SK텔레콤과 함께!',
  "'5G 클러스터'에 대해 안내해 드릴게요."]]

In [51]:
save_path = './data/train.csv'
train_dataset = []
for intent_nm, queries in rep_queries.items():
    for query in queries:
        for negative in negative_dict[intent_nm]:
            #print([query, intent_nm, negative])
            train_dataset.append([query, intent_nm, negative])

with open(save_path, 'a') as f:
    for data in train_dataset:
        line = '\t'.join(data)
        f.write(line + '\n')


In [52]:
save_path = './data/val.csv'
val_dataset = []
for intent_nm, queries in neg_rep_queries.items():
    for query in queries:
        for negative in negative_dict[intent_nm]:
            #print([query, intent_nm, negative])
            val_dataset.append([query, intent_nm, negative])

with open(save_path, 'a') as f:
    for data in val_dataset:
        line = '\t'.join(data)
        f.write(line + '\n')


In [55]:
train_dataset[100:120]

[['로밍안심음성 가입 방법 알려줘', '자동안심 T로밍 음성', '자동안심 T로밍'],
 ['로밍안심음성 가입 방법 알려줘', '자동안심 T로밍 음성', '자동안심 T로밍 데이터'],
 ['로밍안심음성 가입 방법 알려줘', '자동안심 T로밍 음성', '자동안심 T로밍 데이터 해지'],
 ['로밍안심음성 가입 방법 알려줘', '자동안심 T로밍 음성', 'T로밍 음성 사용량 안내 문자 차단'],
 ['안심로밍음성 신청할래', '자동안심 T로밍 음성', '자동안심 T로밍'],
 ['안심로밍음성 신청할래', '자동안심 T로밍 음성', '자동안심 T로밍 데이터'],
 ['안심로밍음성 신청할래', '자동안심 T로밍 음성', '자동안심 T로밍 데이터 해지'],
 ['안심로밍음성 신청할래', '자동안심 T로밍 음성', 'T로밍 음성 사용량 안내 문자 차단'],
 ['로밍안심음성 신청하려고', '자동안심 T로밍 음성', '자동안심 T로밍'],
 ['로밍안심음성 신청하려고', '자동안심 T로밍 음성', '자동안심 T로밍 데이터'],
 ['로밍안심음성 신청하려고', '자동안심 T로밍 음성', '자동안심 T로밍 데이터 해지'],
 ['로밍안심음성 신청하려고', '자동안심 T로밍 음성', 'T로밍 음성 사용량 안내 문자 차단'],
 ['로밍안심음성 신청하고 싶어', '자동안심 T로밍 음성', '자동안심 T로밍'],
 ['로밍안심음성 신청하고 싶어', '자동안심 T로밍 음성', '자동안심 T로밍 데이터'],
 ['로밍안심음성 신청하고 싶어', '자동안심 T로밍 음성', '자동안심 T로밍 데이터 해지'],
 ['로밍안심음성 신청하고 싶어', '자동안심 T로밍 음성', 'T로밍 음성 사용량 안내 문자 차단'],
 ['안심음성로밍 가입할 수 있어', '자동안심 T로밍 음성', '자동안심 T로밍'],
 ['안심음성로밍 가입할 수 있어', '자동안심 T로밍 음성', '자동안심 T로밍 데이터'],
 ['안심음성로밍 가입할 수 있어', '자동안심 T로밍 음성', '자동안심 T로밍 데이터 해지']

In [62]:
train_dataset = []
for intent_nm, queries in rep_queries.items():
    for query in queries:
        for negative in negative_dict[intent_nm]:
            #print([query, intent_nm, negative])
            train_dataset.append(InputExample(texts=[query, intent_nm, negative]))

In [63]:
train_batch_size = 64

In [64]:
from torch.utils.data import DataLoader

In [65]:
train_datasets = SentencesDataset(train_dataset, model)
train_dataloader = DataLoader(train_datasets, shuffle=True, batch_size=train_batch_size)
train_loss = losses.TripletLoss(model=model, triplet_margin = 0.5)

In [66]:
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=10)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/10970 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 30.00 MiB (GPU 0; 31.75 GiB total capacity; 30.15 GiB already allocated; 15.75 MiB free; 30.40 GiB reserved in total by PyTorch)